In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import csv
import pickle
import libs.utils as utils
import libs.handle.dataset as handle_dataset
import spikeinterface.full as si
import spikeinterface.extractors as se
from spikeinterface.preprocessing import get_motion_parameters_preset, get_motion_presets


c:\Users\Sugimoto\Code\tc2025_shysgmt\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  
print(torch.cuda.get_device_name(0))  

2.5.1+cu121
True
NVIDIA GeForce RTX 3050


In [2]:
session_dir = r'Z:\Temp\RSS033_241101_153102'

In [3]:
session = handle_dataset.Session(session_dir)

In [4]:
# Extract dataset of target probe
i_imec = 0
session.add_analyzer_clean(i_imec=i_imec)

In [10]:
sorting_analyzer = session.analyzer_clean

In [11]:
sorting_analyzer.extensions['quality_metrics'].get_data()

,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,amplitude_cutoff,amplitude_median,amplitude_cv_median,amplitude_cv_range
9,17.264712,0.990741,inf,0.085394,495.0,0.079216,294.0,0.065,0.0,-5979.003906,NaN,NaN
26,8.333933,0.981481,inf,0.004442,6.0,0.002224,2.0,0.010,0.0,-5979.003906,NaN,NaN
37,3.271621,1.000000,inf,0.033629,7.0,0.051784,7.0,0.050,0.0,-5979.003906,NaN,NaN
43,1.842524,0.962963,inf,0.060587,4.0,0.095434,4.0,0.055,0.0,-5979.003906,NaN,NaN
55,9.277409,1.000000,inf,0.063328,106.0,0.070599,76.0,0.065,0.0,-5979.003906,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
890,2.316267,1.000000,inf,0.182104,19.0,0.090340,6.0,0.155,0.0,-5979.003906,0.000000e+00,8.166599e-08
895,2.528071,1.000000,inf,0.160914,20.0,0.216542,16.0,0.220,0.0,-5979.003906,8.166600e-08,1.388322e-07
896,1.360913,1.000000,inf,0.055528,2.0,0.042551,1.0,0.150,0.0,-5979.003906,8.166600e-08,1.633320e-07
898,1.562073,1.000000,inf,0.147516,7.0,0.099813,3.0,0.170,0.0,-5979.003906,NaN,NaN
